In [1]:
# autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

import pathmagic
from tools.project import proj_dir

In [2]:
device = "cuda"

In [3]:
from data.video_depth_focus_data import VideoDepthFocusData

def create_dataset(data_type):
    data = VideoDepthFocusData(proj_dir("datasets"), data_type, "art_scene")

    data.configure_blender_data(sample_count=2,
                                sample_skip=0,
                                single_depth_frame=None,
                                single_coc_frame=None,
                                use_allinfocus=False,
                                include_coc=True 
                               )
    
    return data

train_data = create_dataset("train")
val_data = create_dataset("val")

No config found -> Use all

CoC Max 0.0006044905008635579
No config found -> Use all

CoC Max 0.0006044905008635579


In [4]:
from trainer import TrainLoggerTensorboardX

train_logger = TrainLoggerTensorboardX(
                model_id="43",
                model_desc="no_desc",
                log_dir="/home/kevin/Documents/master-thesis/logs/test_log",
                log_it_freq=1,
                log_img_freq=1,
                log_val_freq=1,
                val_loss_func=[],
                save_imgs=True)

[2019-04-21 13:23:32,746] [train_logger.py:394] DEBUG: /home/kevin/Documents/master-thesis/logs/test_log/tblogs/m43_no_desc


In [5]:
from torch import nn
from net.pool_net import PoolNet, PoolNetEncoder, PoolNetDecoder
from net.coc_depth_net import CoCDepthEncShareNet, CoCDepthLayeredNet

"""
model=CoCDepthEncShareNet(enc_sizes=PoolNetEncoder.dft_sizes * 1 // 8,
                          dec_sizes=PoolNetDecoder.dft_sizes * 1 // 8,
                          #dec_pool_layers=[True, True, True, True, False],
                          bn_eps=1e-4,
                          bias=False,
                          act_func=nn.ReLU()).to(device)
"""

model=CoCDepthLayeredNet(enc_sizes=PoolNetEncoder.dft_sizes * 1 // 8,
                         dec_sizes=PoolNetDecoder.dft_sizes * 1 // 8,
                         dec_pool_layers=[True, True, True, True, False],
                         bn_eps=1e-4,
                         bias=False,
                         act_func=nn.ReLU()).to(device)

/home/kevin/miniconda2/envs/torch/lib/python3.6/site-packages/torch/cuda/__init__.py:118: UserWarning: 
    Found GPU0 GeForce GTX 660M which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [6]:
import torch

optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

In [7]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True, num_workers=1)
val_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=False, num_workers=1)

In [8]:
from trainer.trainer import TrainerCoCDepth

trainer = TrainerCoCDepth(model=model,
                          device=device,
                          optimizer=optimizer,
                          logger=train_logger
                         )

trainer.train(train_loader,
              val_loader,
              1,
              checkpoint_freq=9999999)

13:23:34 START
13:23:37 [Epoch 1/1] TRAIN train_loss: 1.9191025
13:23:38 [Epoch 1/1] VAL   train_loss: nan
13:23:38 FINISH


nan